In [2]:
import numpy as np
from scipy.stats import betaprime
from scipy.special import beta
from scipy.signal import find_peaks, peak_prominences
%matplotlib tk
import matplotlib.pyplot as plt
import random
import matplotlib.animation as animation

## Generador:
Clase que contiene los datos para generar señales con parámetros aleatorios.
Posee un método:
### generar_senial
Este método devuelve la señal simulada, con parámetros aleatorios.
Llamando a este método con argumentos adecuados, se puede que se cambien los parámetros del objeto
    

In [3]:
class Generador:
    def __init__(self,picos:list=[1.8, 2.8, 5],prob:list=[0.3, 0.1, 0.6], muestras:int=1000):
        self.picos=picos                        #Valores de tiempo donde se centran los picos
        self.P=prob                             #Probabilidades de ocurrencia de los picos
        self.muestras=muestras
        self.t=np.linspace(0, 10, muestras)     #define los puntos de muestreo
        

    def generar_senial(self, **kwargs):
        ''' Función que genra una señal para un número de muestras dadas,
         entre 0 y 10 micro segundos
         picos -> Lista con los tiempos donde se centran los picos
         prob -> Probabilidad que tienen los picos de ocurrir

         Si estos argumentos no se brindan, se toman los definidos en el objeto
         '''
        
        #Si se declaran los picos o las probabilidades, se cambian en la raiz
        if 'picos' in kwargs: self.picos=kwargs['picos']
        if 'prob' in kwargs: self.P=kwargs['prob'] 
        if 'muestras' in kwargs:
            self.muestras=kwargs['muestras']
            self.t=np.linspace(0, 10, kwargs['muestras'])
        ####################################################################
        ##### A partir de acá se generan las señales base de los picos #####
        ####################################################################

        a,b,sigma_t = 4,2,0.15                  #Seteo los parámetros para la distribución beta prima

        taux=[]                                 #Determmino las posiciones reales de los picos siguiendo una
        for tx in self.picos:                   #distribución beta prima
            t = betaprime.rvs(a, b, loc=tx, scale=sigma_t, size=1)
            taux.append(t[0])
        t0s=np.array(taux)
        
        sigmas=np.array([np.random.normal(0.05, 0.005) for x in self.picos])      #Calculo los anchos de los pulsos(sigmas)    
        omegas=np.array([np.random.normal(5*np.pi, 0.05) for x in self.picos])    #Calculo las frecuencias de oscilacion

        #Probabilidades de los picos
        rx=np.random.random(size=len(self.picos))             #obtengo tres valores random entre 0 y 1
        px=np.array(self.P)                                   #Convierto probabilidades a NP
        aciertos=np.array([r<=p for r,p in zip(rx,px)])       #En los aciertos se establece TRUE si el pico aparece y FALSE si no
    
        #Determino los valres de A
        A = 1 / (sigmas * np.sqrt(2 * np.pi))
        #pulsos = A * np.exp(-(self.t - t0s) ** 2 / (2 * sigmas ** 2)) * np.cos(omegas * (t - t0s + sigmas))
        pulsosparciales =np.array([match*Ax * np.exp(-(self.t - t0) ** 2 / (2 * sigma ** 2)) * np.cos(omega * (self.t - t0 + sigma)) for
                                          Ax, t0,sigma,omega,match in zip(A,t0s,sigmas,omegas,aciertos)])
        pulsos=pulsosparciales.sum(axis=0)
       

        ####################################################################
        ### A partir de acá se generan las señales "accesorio" (ruidos)  ###
        ####################################################################

        #obtengo el ruido absoluto
        ruido_abs = np.random.normal(0, 0.2, size=self.muestras)
        #Obtengo el ruido relativo
        ruido_rel = np.random.normal(1, 0.1, size=self.muestras)
        #Obtengo la señal de fondo
        fondo=(2-self.t) / 100

        ####################################################################
        #### Sumatoria de todas las señales para formar la señal final  ####
        ####################################################################

        pulso=pulsos*ruido_rel+ruido_abs+fondo

        return self.t, pulso
    

    
    def muestrear(self,nmed=10000, picos:list=[1.8, 2.8, 5],prob:list=[0.3, 0.1, 0.6]):
        self.picos=picos
        self.prob=prob

        dataset=np.empty((nmed,self.muestras))
        for x in range(nmed):
            senial=self.generar_senial(picos=picos, prob=prob)[1]
            dataset[x]=senial
        self.dataset= dataset

        return self.t, dataset.sum(axis=0)/nmed





#### Creo el objeto que genera las señales
muestras indica  el muestreo de tiempo entre 0  y 10 $\mu$ Segundos (en este caso 5000)

In [4]:
sim=Generador(muestras=5000)

#### Animo el objeto

# Animación
fig, ax = plt.subplots()
ax.set_xlim(0, 10)
ax.set_ylim(-5, 15)
ax.set_xlabel('Tiempo [μS]')
ax.set_ylabel('Tensión [mV]')
ax.grid(color='m')
line, = plt.plot([], [])


def animate(i,L):
    t,sig=sim.generar_senial()
    L.set_data(t, sig)
    return L,

repeticiones=100
animacion=animation.FuncAnimation(fig, animate,repeticiones,fargs=(line,), interval=200, blit=True)
plt.show()

In [5]:
x=[0,0,0]
y=[0,0,0]
maximos=[0,0,0]
medizq=[0,0,0]
medder=[0,0,0]

x[0],y[0]=sim.muestrear(nmed=10000, picos=[1.8,2.8,5], prob=[1,0,0])
x[1],y[1]=sim.muestrear(nmed=10000, picos=[1.8,2.8,5], prob=[0.8,0.1,0.1])
x[2],y[2]=sim.muestrear(nmed=10000, picos=[1.8,2.8,3.5], prob=[0.85,0.1,0.05])

def filtroprom(x):
    if x>0.02: return 1
    else: return 0

def filtromax(max):
    return max>0

In [6]:
for n, (xn ,yn) in enumerate(zip(x,y)):
    maximos[n], prop = find_peaks(yn,prominence=0.02, width=50)
    medizq[n]=prop["left_ips"]
    medder[n]=prop["right_ips"]

In [30]:
fig, axs = plt.subplots(3, 1,figsize=(8, 10))
titulos=["Picos en t=1.8,2.8,5 y probabilidades 1,0,0","Picos en t=1.8,2.8,5 y probabilidades 0.8,0.1,0.1", "Picos en t=1.8,2.8,3.5 y probabilidades 0.85,0.1,0.05"]

for rep,(xn,yn,mxn,titx) in enumerate(zip (x,y,maximos,titulos)):

    axs[rep].grid(linestyle=':',which='minor',alpha=0.2, color='gray')
    axs[rep].grid(linestyle=':',which='major',alpha=0.6, color='black')
    
    axs[rep].plot(xn,yn, label='Promedios')
    axs[rep].plot(mxn*10/sim.muestras,yn[mxn],'r*',label='Máximos')
    medios=(np.rint(np.append(medizq[rep],medder[rep]))).astype(int)
    axs[rep].plot(medios*10/sim.muestras,yn[medios],'gd',label='Medios')
    axs[rep].set_title(titx)
    axs[rep].set_xlabel('Tiempo ($\mu$ segundos)')
    axs[rep].set_ylabel('Tensión (mV)')

    axs[rep].set_ylim([-0.2,1.4])

    for n,m in enumerate(mxn):
        text="max {}: {:.3}".format(n+1,yn[m])
        axs[rep].annotate(text,xy= (m*10/sim.muestras-0.2,yn[m]+0.05))
    for n,(i,d) in enumerate(zip(medizq[rep],medder[rep])):
        ancho=(d-i)*10/sim.muestras
        text="ancho {}: {:.3} $\mu$S".format(n+1,ancho)
        axs[rep].annotate(text,xy= (5,1.2-0.15*n))
    axs[rep].minorticks_on()
    axs[rep].legend(loc="best", title="Referencias")
    
    


# Ajustar los márgenes de las subgráficas
fig.tight_layout()
plt.show()

In [206]:
peaks, properties = find_peaks(y[2], prominence=0.02, width=20)
iad=np.append(properties["left_ips"],properties["right_ips"])
riad = (np.rint(iad)).astype(int)



In [208]:
plt.plot(y[2])

plt.plot(peaks, y[2][peaks], "x")
plt.plot(riad,y[2][riad], "d")

##plt.vlines(x=peaks, ymin=x[peaks] - properties["prominences"], ymax = x[peaks], color = "C1")

#plt.hlines(y=properties["width_heights"], xmin=properties["left_ips"],xmax=properties["right_ips"], color = "C1")

plt.show()